# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

In this lab you will: 

- Use ANOVA for testing multiple pairwise comparisons 
- Interpret results of an ANOVA and compare them to a t-test

## Load the data

Start by loading in the data stored in the file `'ToothGrowth.csv'`: 

In [1]:
# Your code here
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

df = pd.read_csv('ToothGrowth.csv')
df.head()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


## Generate the ANOVA table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage:  

In [2]:
# Your code here
model = ols('len ~ C(supp) + C(dose)', data=df).fit()

anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

               sum_sq    df          F        PR(>F)
C(supp)    205.350000   1.0  14.016638  4.292793e-04
C(dose)   2426.434333   2.0  82.810935  1.871163e-17
Residual   820.425000  56.0        NaN           NaN


## Interpret the output

Make a brief comment regarding the statistics and the effect of supplement and dosage on tooth length: 

In [3]:
# Your comment here
"""
The sum square represents the variability attributed to each factor. 
For instance, the sum of c(supp) is 205.35 explaining the degree of tooth length explained by the type of supplement, i.e; OJ verses VC.
The sum of squares in c(dose) is high. This means a larger amount of variance in tooth length explained by different supplement dosages.
The residual sum of squares represents variance in the tooth length that it is not.
"""

'\nThe sum square represents the variability attributed to each factor. \nFor instance, the sum of c(supp) is 205.35 explaining the degree of tooth length explained by the type of supplement, i.e; OJ verses VC.\nThe sum of squares in c(dose) is high. This means a larger amount of variance in tooth length explained by different supplement dosages.\nThe residual sum of squares represents variance in the tooth length that it is not.\n'

## Compare to t-tests

Now that you've had a chance to generate an ANOVA table, its interesting to compare the results to those from the t-tests you were working with earlier. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterward, you'll conduct a t-test to compare the tooth length of these two different samples: 

In [4]:
# Your code here
oj_sample = df[df['supp'] == 'OJ']['len']
vc_sample = df[df['supp'] == 'VC']['len']

print(oj_sample.head())
print(vc_sample.head())

30    15.2
31    21.5
32    17.6
33     9.7
34    14.5
Name: len, dtype: float64
0     4.2
1    11.5
2     7.3
3     5.8
4     6.4
Name: len, dtype: float64


Now run a t-test between these two groups and print the associated two-sided p-value:

In [5]:
# Calculate the 2-sided p-value for a t-test comparing the two supplement groups
from scipy import stats

t_test, p_value = stats.ttest_ind(oj_sample, vc_sample)
print(t_test)
print(p_value)


1.91526826869527
0.06039337122412849


## A 2-Category ANOVA F-test is equivalent to a 2-tailed t-test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [8]:
# Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
# Compare the p-value to that of the t-test above. 
# They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)
model = ols('len ~ supp', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_p_value = anova_table['PR(>F)']['supp']
print(anova_p_value)
print(anova_table)


0.060393371224128745
               sum_sq    df         F    PR(>F)
supp       205.350000   1.0  3.668253  0.060393
Residual  3246.859333  58.0       NaN       NaN


## Run multiple t-tests

While the 2-category ANOVA test is identical to a 2-tailed t-test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [9]:
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2-sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [10]:
# Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
# for all combinations of the supplement-dose groups listed above. 
# (Since there isn't a control group, compare each group to every other group.)

import itertools
groups = df.groupby(['supp', 'dose'])
group_combinations = list(itertools.combinations(groups, 2))
for (group1_name, group1), (group2_name, group2) in group_combinations:
    t_stat, p_val = stats.ttest_ind(group1['len'], group2['len'], equal_var=False)
    print(f"Comparison between {group1_name} and {group2_name}: p-value = {p_val:.5f}")

Comparison between ('OJ', 0.5) and ('OJ', 1.0): p-value = 0.00009
Comparison between ('OJ', 0.5) and ('OJ', 2.0): p-value = 0.00000
Comparison between ('OJ', 0.5) and ('VC', 0.5): p-value = 0.00636
Comparison between ('OJ', 0.5) and ('VC', 1.0): p-value = 0.04601
Comparison between ('OJ', 0.5) and ('VC', 2.0): p-value = 0.00001
Comparison between ('OJ', 1.0) and ('OJ', 2.0): p-value = 0.03920
Comparison between ('OJ', 1.0) and ('VC', 0.5): p-value = 0.00000
Comparison between ('OJ', 1.0) and ('VC', 1.0): p-value = 0.00104
Comparison between ('OJ', 1.0) and ('VC', 2.0): p-value = 0.09653
Comparison between ('OJ', 2.0) and ('VC', 0.5): p-value = 0.00000
Comparison between ('OJ', 2.0) and ('VC', 1.0): p-value = 0.00000
Comparison between ('OJ', 2.0) and ('VC', 2.0): p-value = 0.96385
Comparison between ('VC', 0.5) and ('VC', 1.0): p-value = 0.00000
Comparison between ('VC', 0.5) and ('VC', 2.0): p-value = 0.00000
Comparison between ('VC', 1.0) and ('VC', 2.0): p-value = 0.00009


## Summary

In this lesson, you implemented the ANOVA technique to generalize testing methods to multiple groups and factors.